In [101]:
import ipywidgets as widgets
from ipywidgets import *
from PIL import Image
import fastai.vision
from fastai.basic_train import load_learner
import torch
import pandas as pd
import numpy as np
from torchvision.models import vgg16_bn
import torch.nn as nn
from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from fastai.callbacks import *

In [78]:
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [79]:
from pathlib import Path
Path = Path()
learn_inf = load_learner(Path)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
btn_run = widgets.Button(description="generate")
in_pl = widgets.Output()

In [124]:
def on_click_generate(change):
    for name, file_info in btn_upload.value.items():
        bts = io.BytesIO(file_info['content'])
        img = open_image(bts)
        
    out_pl.clear_output()
    pred, pred_ioidx, probs = learn_inf.predict(img)
    with out_pl: display(pred)
        
def on_upload(change):
    for name, file_info in btn_upload.value.items():
        bts = io.BytesIO(file_info['content'])
        img = open_image(bts)
        
    in_pl.clear_output()
    with in_pl: display(img.resize(torch.Size([img.shape[0],256,256])))
btn_upload.observe(on_upload)
btn_run.on_click(on_click_generate)

In [128]:
display(VBox([widgets.Label('Upload a medical image with text!'),
     btn_upload, in_pl, btn_run, out_pl]))